In [7]:
import pandas as pd

In [9]:
teste = pd.read_csv('../data_science/turmas_2015-2017.csv')

series_teste = (teste['ano'].map(str)+teste['periodo'].map(str))
teste['periodoano'] = series_teste.astype(int)

teste.to_csv('../data_science/turmas_new.csv')

In [10]:
dataFrame = pd.read_csv('../data_science/turmas_new.csv')

discA = 'CÁLCULO I'
discB = 'CÁLCULO II'

dataFrame = dataFrame.dropna()

print(dataFrame.columns)

Index(['Unnamed: 0', 'Unnamed: 0.1', 'descricao', 'discente', 'id_turma',
       'media_final', 'codigo', 'nome', 'ano', 'periodo',
       'unidade_responsavel', 'id_curso', 'periodoano'],
      dtype='object')


In [96]:
def correlacao(discA, discB):
    dataFrame = pd.read_csv('data_science/df_disciplinas2015New.csv')
    dataFrameA = dataFrame[dataFrame['nome'] == discA]
    dataFrameB = dataFrame[dataFrame['nome'] == discB]

    # filtrando apenas alunos que pagaram ambas as disciplinas
    # series_discentesA = dataFrameA.discente.unique()
    # series_discentesB = dataFrameB.discente.unique()

    # dataFrameA = dataFrameA[dataFrameA['discente'].isin(series_discentesA)]
    # dataFrameB = dataFrameB[dataFrameB['discente'].isin(series_discentesB)]

    # Aprovados no DiscA
    dataFrameA = dataFrameA[dataFrameA['descricao'].str.contains('APROVADO')]

    series_aprovados = dataFrameA.discente.unique()

    df_finalB = pd.DataFrame()

    for dis in series_aprovados:
        linhas = dataFrameB[dataFrameB['discente'] == dis]
        linha = linhas[linhas['periodoano'] == linhas.periodoano.min()]

        df_finalB = pd.concat([df_finalB, linha])

    # concatenando das tabelas
    colunas = ['discente', 'media_final', 'nome']
    dataFrameA = dataFrameA[colunas]
    df_finalB = df_finalB[colunas]

    conc = pd.concat([dataFrameA, df_finalB])

    df = pd.crosstab(conc.discente, conc.nome, conc.media_final,aggfunc=np.mean)
    df = df.dropna()
    df_correlacao = df.corr()

    return df_correlacao[discA][discB]

In [109]:
# FUNÇÃO DO GRÁFICO DAS COORDENADAS
disciplinas = ['CÁLCULO I', 'CÁLCULO II', 'CÁLCULO III']
dataFrame = dataFrame[dataFrame['nome'].isin(disciplinas)]
display(dataFrame.head())
# e o caso de mais de uma nota?
teste = dataFrame.groupby(['discente']).nome.count()
teste.head(10)
# df = pd.crosstab(dataFrame.discente, dataFrame.nome, dataFrame.media_final,aggfunc=np.mean)
# df.head()

,Unnamed: 0,Unnamed: 0.1,index,descricao,discente,id_turma,media_final,codigo,nome,ano,periodo,unidade_responsavel,id_curso,periodoano
77,77,77,721313,CANCELADO,3d688dbe74a84992ce43181e08c0f295,1242602,0.0,ECT2103,CÁLCULO I,2015,1,ESCOLA DE CIÊNCIAS E TECNOLOGIA,10320810.0,20151
78,78,78,721316,REPROVADO,6c4acceb9b1288b61d893dc1b5005764,1242602,0.0,ECT2103,CÁLCULO I,2015,1,ESCOLA DE CIÊNCIAS E TECNOLOGIA,10320810.0,20151
79,79,79,721319,APROVADO,bb67c2ca87d0e1fcf027b2f0f6df4221,1242602,8.5,ECT2103,CÁLCULO I,2015,1,ESCOLA DE CIÊNCIAS E TECNOLOGIA,10320810.0,20151
80,80,80,721322,REPROVADO,9897eefc366f1f546e5340390c3df0cd,1242602,1.3,ECT2103,CÁLCULO I,2015,1,ESCOLA DE CIÊNCIAS E TECNOLOGIA,10320810.0,20151
81,81,81,721331,REPROVADO,859abe9fcb5769f73f54cba6592b81af,1242602,1.7,ECT2103,CÁLCULO I,2015,1,ESCOLA DE CIÊNCIAS E TECNOLOGIA,10320810.0,20151


discente
0010345ff1be7da25478b5a96c772616    5
0015133009a53469a2615e7dfa6d638a    1
005cdf6ddf92e14dfa387c87f3f6e946    3
00a0c8929d0b1b7c985d5473a20877fd    1
00a54c3ed49fdd50c7dd248458255e17    5
00b24210514efe9ee169982514b9d1ed    1
00b4c9b8bd631422f6a05bad05774d2b    2
00c95b0e427b18e5443272c0c01549c3    3
00d4dab41b0960fd67973cbe407130c8    1
00d67b98ad2471688f50b37e26017c8e    2
Name: nome, dtype: int64